# Evaluating Siamese GNNs on Synthetic Graph Edit Distance Tasks

## Initialize and load model

We first load a particular checkpoint of interest.

In [20]:
from model import GraphMatchingNetwork, GraphConvolutionNetwork
import torch

# Load checkpoint (full)
filename = f'./checkpoints/gcn_frosty-tree-7.pth.tar'
checkpoint = torch.load(filename, map_location=lambda storage,
                        loc: storage.cuda() if torch.cuda.is_available() else storage.cpu())

# Retrieve state dict
state_dict = checkpoint['state_dict']
cfg = checkpoint['config']

# Initialize model
if cfg.model.name == 'gmn':
    model = GraphMatchingNetwork(cfg)
elif cfg.model.name == 'gcn':
    model = GraphConvolutionNetwork(cfg)
else: raise ValueError("Model should be either GMN or GCN")

# Load state dict
model.load_state_dict(state_dict)  # should output "<All keys matched successfully>"

<All keys matched successfully>

## Fixed Triplet Dataset

In [7]:
from utils import load_pickle

# Fixed dataset settings
num_nodes = 10
kp = 1
kn = 2
pe = 0.2

# Load dataset
dataset = load_pickle(f"./data/FixedDatasetGED_nodes={num_nodes}_kp={kp}_kn={kn}_pe={pe}.pickle")

## Triplet accuracy

Here we compute the triplet accuracy. </br>


In [21]:
from utils import reshape_and_split_tensors
from loss import triplet_loss
from metrics import euclidean_distance
from tqdm.auto import tqdm
import numpy as np

# Set model to eval model
model.eval()

for num_nodes in [10, 15, 20, 30, 50]:

    # Fixed dataset settings
    kp = 1
    kn = 2
    pe = 0.2

    # Load dataset
    dataset = load_pickle(f"./data/FixedDatasetGED_nodes={num_nodes}_kp={kp}_kn={kn}_pe={pe}.pickle")
    dataset = dataset[:100]

    # Loop through data
    rel_distances = []
    losses = []
    for triplet in tqdm(dataset, total=len(dataset)):

        # Prepare the data
        edge_index = triplet['edge_index']  # edge index
        node_feats = torch.ones(triplet.num_nodes, cfg.model.node_dim)  # node features to all-ones
        edge_feats = torch.ones(triplet.num_edges, cfg.model.edge_dim)  # edge features to all-ones
        batch_id = triplet['order']

        # Feedforward
        _, graph_feats = model(edge_index, x1=node_feats, x2=None, edge_feats=edge_feats, batch=batch_id)
        graph_feats = reshape_and_split_tensors(graph_feats, 4)

        # Get accuracy measure
        rel_distances.append(euclidean_distance(graph_feats[0], graph_feats[1]) \
                   - euclidean_distance(graph_feats[2], graph_feats[3]))

        # Get loss
        losses.append(triplet_loss(*graph_feats, cfg).detach().numpy())

    # print(rel_distances)
    losses = np.mean(losses)
    corrects = torch.sum(torch.tensor(rel_distances) < 0)
    accuracy = corrects / len(dataset)

    print(f"Performance (nodes={num_nodes} kp={kp} kn={kn} pe={pe}):\n"
          f"\tAccuracy:\t{accuracy}\n"
          f"\tLosses:\t\t{losses}\n")

  0%|          | 0/100 [00:00<?, ?it/s]

Performance (nodes=10 kp=1 kn=2 pe=0.2):
	Accuracy:	1.0
	Losses:		0.0



  0%|          | 0/100 [00:00<?, ?it/s]

Performance (nodes=15 kp=1 kn=2 pe=0.2):
	Accuracy:	0.009999999776482582
	Losses:		120.17671203613281



  0%|          | 0/100 [00:00<?, ?it/s]

Performance (nodes=20 kp=1 kn=2 pe=0.2):
	Accuracy:	0.0
	Losses:		229.39964294433594



  0%|          | 0/100 [00:00<?, ?it/s]

Performance (nodes=30 kp=1 kn=2 pe=0.2):
	Accuracy:	0.0
	Losses:		176.78652954101562



  0%|          | 0/100 [00:00<?, ?it/s]

Performance (nodes=50 kp=1 kn=2 pe=0.2):
	Accuracy:	0.5299999713897705
	Losses:		0.8321780562400818



**Looks weird ?? Why are the scores so off?**

We will try it using the other dataloader